In [8]:
# Install required packages to connect to google sheets
!pip install gspread oauth2client df2gspread

  Created wheel for df2gspread: filename=df2gspread-1.0.4-py3-none-any.whl size=11958 sha256=591c45cbd42ae547c7be0940e418553e77ed3b7e5da27be5aafd3846e6338684
  Stored in directory: c:\users\v.perez\appdata\local\pip\cache\wheels\4d\78\20\cf9a2810f93885ad71bd351c8b604455ba160ac4b0e9b7d526
Successfully built df2gspread


In [6]:
import gspread 
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://docs.google.com/spreadsheets/d/1auoQ9XanosnM7RUInzqeZi9EIgwtCtmtubNpXrfF6OM/edit#gid=1038370816'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('./covid19-sentanalysis-10fda0e571ef.json', scope) 
gc = gspread.authorize(credentials)

In [10]:
# Load twitter credentials
with open("covid19-sentanalysis-twitter_credentials.json") as datafile:
  data = json.load(datafile)

In [12]:
import tweepy
from textblob import TextBlob


# Step 1 - Authenticate
consumer_key= data['consumer_key'] #'API_CONSUMER_KEY_HERE'
consumer_secret=  data['consumer_secret']#'CONSUMER_SECRET_HERE'

access_token= data['access_token_key'] #'ACCESS_TOKEN_HERE'
access_token_secret= data['access_token_secret'] #'ACCESS_TOKEN_SECRET_HERE'



auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

#print("API")

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

home_tweets = api.home_timeline(count=10)
print("printing tweets from timeline \n ")
for tweet in home_tweets:
    print(tweet.text)
    print("")

Authentication OK
printing tweets from timeline 
 
Curiosamente, esos que dividen entre buenos y malos suelen ser los peores.

Desde el Concejo buscan acompañar el pedido para exceptuar a surfistas profesionales
https://t.co/No4hDP8nJL

If you're on PC, the Logitech G Pro X is one the best wired gaming headsets around. If you're on console, not so mu… https://t.co/8Fk1LjOssQ

RT @los_fatales: "puede venirse el mundo abajo" Flopa ♥️ https://t.co/I5jPIeyjXx

Jaime Sabines y la importancia de llegar a tiempo. https://t.co/fhbzF6Ad4c

RT @Tutehumor: #Tute https://t.co/oqL7syjetk

despertar y leer mensajes lindos que no leí anoche porque me quedé dormida en medio de un montón de mensajes lindos.

y o  y a  g a n é.

💕.

La tienda del barrio, la librería de toda la vida. Hagamos realidad una cadena de apoyo.… https://t.co/xt36R3sAV9

RT @isabelle_ax: Wrote a piece for the new issue of TIME about activism, hope and isolation during COVID19. 
https://t.co/IYeRVYmTKd https:…



**GET Twitter Stream and Do Sentiment Analysis in Real time**


In [2]:
from tweepy import Stream
from tweepy import StreamListener
#from google.colab import files
import pandas as pd
import json
import re
import csv
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\v.perez\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
trump = 0
warren = 0

header_name = ['Trump', 'Warren','Text','id', 'created_at', 'user_id','coordinates','location']
with open('sentiments.csv', 'w') as file:
  writer = csv.DictWriter(file, fieldnames = header_name)
  writer.writeheader()

# sentiments_df = pd.DataFrame(columns=['Trump','Warren'])
#api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

class Listener(StreamListener):
  def __init__(self):
    super().__init__()
    self.max_tweets = 10
    self.tweet_count = 0
    
  def on_data(self, data):
    raw_twitts = json.loads(data)
    try:
      #  Fields we need: id, created_at, text, coordinates, author_id
      full_tweets = raw_twitts.copy()
   
      # TO DO: we must drop from full_tweets the tweets that consist only on RT, numbers, etc (see regex used below)
      
            
      tweets = raw_twitts['text']
      tweets = ' '.join(re.sub("(@[A-Za-z0-9]+) | ({*0-9A-Za-z \t]) |] (\wt:\/\/\St+)", " ", tweets).split())
      tweets = ' '.join(re.sub('RT',' ', tweets).split())  
        
      #tweets = raw_twitts['text']
      #tweets = ' '.join(re.sub("(@[A-Za-z0-9]+) | ({*0-9A-Za-z \t]) |] (\wt:\/\/\St+)", " ", tweets).split())
      #tweets = ' '.join(re.sub('RT',' ', tweets).split())
      blob = TextBlob(tweets.strip())
      global trump
      global warren

      trump_sentiment = 0
      warren_sentiment = 0

      for sent in blob.sentences:
        if "Trump" in sent and "Warren" not in sent:
          trump_sentiment = trump_sentiment + sent.sentiment.polarity
        else:
          warren_sentiment = warren_sentiment + sent.sentiment.polarity

      trump = trump + trump_sentiment
      warren = warren + warren_sentiment
        
        
    
      with open('sentiments.csv', 'a') as file:
        writer = csv.DictWriter(file, fieldnames=header_name)
        info = {'Trump': trump,'Warren': warren, 'Text':raw_twitts['text']
               ,'id':raw_twitts['id'], 'created_at':raw_twitts['created_at']
               , 'user_id':raw_twitts['user']['id'], 'coordinates':raw_twitts['coordinates']
                , 'location':raw_twitts['user']['location']
               }
        writer.writerow(info)

      # sentiments_df['Trump'].append(trump)
      # sentiments_df['Warren'].append(warren)
      # sentiments_df.to_csv('sentiments.csv') 
      print (tweets,'\n')    
    except:
      print('ERROR got')
    else:
      self.tweet_count+=1
      if(self.tweet_count==self.max_tweets):
        print("completed")
        return(False)
      else:
       decoded = json.loads(data)

  def on_error(self, status):
    print(status)

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [6]:
twitter_stream = Stream(auth, Listener())
twitter_stream.filter(track = ['Trump','Warren'])

@DetroitSteel_: Do you think Trump ever set foot in a church before he was elected? 

Biden is not a good candidate but he is not the devil. The faustian bargain was made i… https://t.co/dpnkRkL14K 

@dandrezner: This is your occasional reminder that decent politicians apologize when they cross a line in their discourse. https://t.co/… 

ERROR got
@RepMcGovern: If Beijing violates its int’l commitments &amp; imposes its draconian security law on #HongKong then it is time for the Trump… 

@edstetzer: New: "Pres. Trump, Governors, &amp; Churches: We Don't Need an Immediate Opening, But We Do Need Communication &amp; Collaboration."… 

@africatechie: Funny how when the drug was being administered to millions of Africans as a malaria drug, these risks didn't seem to both… 

Trump admin considered 1st NUCLEAR TEST since 1992 as show of strength to Russia &amp; China – reports https://t.co/CObmAQrJfo 

@iamwandasykes: Biden feels at home speaking to the Black community. He made a joke. Comed